___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [59]:
import requests

base = {}

r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/dollar_info?json').json()
print(r)

[{'r030': 840, 'txt': 'Долар США(довідковий)', 'rate': 27.9713, 'cc': 'USD', 'exchangedate': '16.04.2021'}]


2. Отримати курс долара за останній рік.

In [60]:
import datetime

today = datetime.date.today()
DB = {}

for i in range(0, 366):
    day = today - datetime.timedelta(i)
    params    = {'date' : day.strftime('%Y%m%d')}
    
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&json', params).json()
    
    DB[day] = r[0]['rate']
print(DB)

{datetime.date(2021, 4, 18): 27.9592, datetime.date(2021, 4, 17): 27.9592, datetime.date(2021, 4, 16): 27.9592, datetime.date(2021, 4, 15): 27.9765, datetime.date(2021, 4, 14): 28.0156, datetime.date(2021, 4, 13): 27.9335, datetime.date(2021, 4, 12): 27.9094, datetime.date(2021, 4, 11): 27.9768, datetime.date(2021, 4, 10): 27.9768, datetime.date(2021, 4, 9): 27.9768, datetime.date(2021, 4, 8): 27.8923, datetime.date(2021, 4, 7): 27.8384, datetime.date(2021, 4, 6): 27.939, datetime.date(2021, 4, 5): 27.9555, datetime.date(2021, 4, 4): 27.8324, datetime.date(2021, 4, 3): 27.8324, datetime.date(2021, 4, 2): 27.8324, datetime.date(2021, 4, 1): 27.8226, datetime.date(2021, 3, 31): 27.8852, datetime.date(2021, 3, 30): 27.9694, datetime.date(2021, 3, 29): 27.9679, datetime.date(2021, 3, 28): 27.9698, datetime.date(2021, 3, 27): 27.9698, datetime.date(2021, 3, 26): 27.9698, datetime.date(2021, 3, 25): 27.8706, datetime.date(2021, 3, 24): 27.7295, datetime.date(2021, 3, 23): 27.6871, datetime.d

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [64]:
import statistics

mean  = {} 
stdev = {} 

for months in range (1,13):
    monthly_rate  = [DB[i] for i in DB.keys() if i.month == months]
    mean[months]  = statistics.mean(monthly_rate)
    stdev[months] = statistics.stdev(monthly_rate)
    
for el in mean.items():
    print(f'Average value for {el[0]} month equals {el[1]}')

print(
)

for el in stdev.items():
    print(f'Standart deviation for {el[0]} month equals {el[1]}')

Average value for 1 month equals 28.219770967741937
Average value for 2 month equals 27.884660714285715
Average value for 3 month equals 27.79555806451613
Average value for 4 month equals 27.571441935483872
Average value for 5 month equals 26.814329032258065
Average value for 6 month equals 26.707343333333334
Average value for 7 month equals 27.31283870967742
Average value for 8 month equals 27.52141935483871
Average value for 9 month equals 27.97671666666667
Average value for 10 month equals 28.324951612903227
Average value for 11 month equals 28.310153333333332
Average value for 12 month equals 28.16971935483871

Standart deviation for 1 month equals 0.1231532113359842
Standart deviation for 2 month equals 0.10631085913415092
Standart deviation for 3 month equals 0.11551466219833594
Standart deviation for 4 month equals 0.42447741893155594
Standart deviation for 5 month equals 0.13470483335438327
Standart deviation for 6 month equals 0.07284717438646436
Standart deviation for 7 month

4. Дану інформацію записати у файл за допомогою pickle.

In [66]:
import pickle

with open('kurs_usd.pickle', 'wb') as f:
    pickle.dump(DB, f)
    pickle.dump(mean, f)
    pickle.dump(stdev, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [168]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
          }
BASE = {}

In [170]:
import requests
from bs4 import BeautifulSoup

BASE = {}
for page in range (1,4): # можна додати ще один конетк, щоб спочатку визначати кількість сторінок
    SITE = f'https://smallpacking.agrosem.ua/shop/page/{page}/'

    r = requests.get(SITE, headers = HEADERS)


    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        for el in (soup.find_all('a',class_="more-detail")):
            el_r = requests.get(el.get('href'), headers = HEADERS)
            lower_soup = BeautifulSoup(el_r.text, 'lxml')
            for n in lower_soup.find_all('h2'):
                if n.text == 'Меню' or n.text == 'Дякуємо!':
                    pass
                else:
                    name = n.text #name

            for s in lower_soup.find_all('span', class_='sku'):
                SKU = s.text[5:] #SKU

            for p in lower_soup.find_all('div', class_='price-container'): 
                l_price = p.text.split('грн')
                price = int(l_price[0].replace(' ','')) #price

            for w in lower_soup.find_all('span', class_='type-packaging-name'):
                if w.text == "Коробка":
                    weight = 1
                elif w.text == "Мішок":
                    weight = 3
                else:
                    weight = 10
            BASE[SKU] = [name, weight, price]
print(BASE)

{'439226': ['Добриво для фруктових та горіхових дерев', 3, 415], '1297': ['Добриво для ягідних чагарників', 3, 415], '91897': ['Добриво для ягідних чагарників', 1, 173], '61169': ['Добриво універсальне грунтове', 1, 173], '6651824': ['Добриво універсальне грунтове', 3, 415], '5227876': ['Добриво універсальне грунтове', 10, 1149], '32237': ['Добриво для декоративних листових', 3, 415], '4165215': ['Добриво для вічнозелених дерев та кущів', 10, 1149], '682645': ['Добриво універсальне грунтове', 3, 415], '7435': ['Добриво універсальне грунтове', 1, 173], '9354141': ['Добриво для газону', 3, 415], '1955': ['Добриво для вічнозелених дерев та кущів', 1, 173], '455799': ['Добриво для газону', 1, 173], '991645': ['Добриво для декоративних листових', 1, 173], '17477': ['Добриво універсальне грунтове', 10, 1149], '7143174': ['Добриво для газону', 3, 415], '3593': ['Добриво для полуниці', 1, 173], '4738184': ['Добриво для вічнозелених дерев та кущів', 3, 415], '3177412': ['Добриво для фруктових т

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [182]:
import sqlite3

con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master")
info = cur.fetchall()
print(info)
con.close()

[('Customers',), ('sqlite_autoindex_Customers_1',), ('OrderItems',), ('sqlite_autoindex_OrderItems_1',), ('Orders',), ('sqlite_autoindex_Orders_1',), ('Products',), ('sqlite_autoindex_Products_1',), ('Vendors',), ('sqlite_autoindex_Vendors_1',)]


2. Виведіть перелік всіх таблиць.

In [183]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
info = cur.fetchall()
print(info)
con.close()

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [234]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("SELECT cust_id FROM Customers")
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('1000000001',)
('1000000002',)
('1000000003',)
('1000000004',)
('1000000005',)


4. Всю таблицю Customers table

In [233]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("SELECT * FROM Customers")
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


5. Список клієнтів (cust_names) відсортованих від Z до A

In [232]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("SELECT distinct cust_name FROM Customers ORDER BY cust_name DESC")
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('Village Toys',)
('The Toy Store',)
('Kids Place',)
('Fun4All',)


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [231]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT Orders.cust_id, Orders.order_num FROM Customers 
            INNER JOIN Orders ON Customers.cust_id = Orders.cust_id""")
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('1000000001', 20005)
('1000000003', 20006)
('1000000004', 20007)
('1000000005', 20008)
('1000000001', 20009)


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [230]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT sum(quantity), item_price  FROM OrderItems GROUP BY order_item
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

(425, 5.49)
(465, 10.99)
(370, 11.99)
(110, 2.99)
(60, 4.49)


8. Товар (з таблиці Products), ціна якого становить 9.49

In [229]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT * FROM Products WHERE prod_price = '9.49'
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('RYL01', 'FNG01', 'King doll', 9.49, '12 inch king doll with royal garments and crown')
('RYL02', 'FNG01', 'Queen doll', 9.49, '12 inch queen doll with royal garments and crown')


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [228]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT prod_name, prod_price 
            FROM Products 
            WHERE prod_price BETWEEN 3 and 9
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('8 inch teddy bear', 5.99)
('12 inch teddy bear', 8.99)
('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)


10. Кількість товару, що було продано

In [226]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT sum(quantity)
            FROM OrderItems
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

(1430,)


11. Кількість найменувань товару, ціна якого більша за 4

In [225]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT count(prod_name)
            FROM Products WHERE prod_price>4
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

(6,)


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [224]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT prod_name, prod_price
            FROM Products
            ORDER BY prod_price DESC
            LIMIT 3
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('18 inch teddy bear', 11.99)
('King doll', 9.49)
('Queen doll', 9.49)


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [237]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT t.cust_id, t.qty
            FROM (SELECT cust_id, count(order_num) as qty
                  FROM Orders
                  GROUP BY cust_id) as t
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()




('1000000001', 2)
('1000000003', 1)
('1000000004', 1)
('1000000005', 1)


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [223]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT Customers.cust_name, Orders.order_num
            FROM Customers
            INNER JOIN Orders
            ON Customers.cust_id = Orders.cust_id
            
            """)
info = cur.fetchall()
for i in info:
    print(i)
con.close()

('Village Toys', 20005)
('Fun4All', 20006)
('Fun4All', 20007)
('The Toy Store', 20008)
('Village Toys', 20009)


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [221]:
con = sqlite3.connect('tysql.sqlite')
cur = con.cursor()
cur.execute("""SELECT prod_name, prod_price 
            FROM Products
            WHERE prod_price<5
            UNION ALL
            SELECT prod_name, prod_price 
            FROM Products
            WHERE prod_price>=5
            """)
info = cur.fetchall()
for i in info:
    print(i)
print(info)
con.close()

('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)
('12 inch teddy bear', 8.99)
('18 inch teddy bear', 11.99)
('King doll', 9.49)
('Queen doll', 9.49)


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


1)  SELECT Product.model, PC.speed, PC.hd FROM Product
    LEFT JOIN PC on Product.model = PC.model
    WHERE PC.price < 500

2)  SELECT DISTINCT(Product.maker) FROM Product
    INNER JOIN Printer on Product.model = Printer.model

3)  SELECT model, ram, screen FROM Laptop
    WHERE price > 1000
    
4)  SELECT * FROM printer WHERE color = 'y'


5)  SELECT model, speed, hd FROM PC 
    WHERE price < 600 and (cd = '12x' or cd = '24x')

6)  SELECT Product.maker, Laptop.speed FROM Product 
    LEFT JOIN Laptop on Product.model = Laptop.model
    WHERE Laptop.hd>=10

7)  SELECT DISTINCT(PC.model), PC.price FROM Product
    INNER JOIN PC on Product.model = PC.model
    WHERE Product.maker = 'B'
    UNION ALL
    SELECT DISTINCT(Laptop.model), Laptop.price FROM Product
    INNER JOIN Laptop on Product.model = Laptop.model
    WHERE Product.maker = 'B'
    UNION ALL
    SELECT DISTINCT(Printer.model), Printer.price FROM Product
    INNER JOIN Printer on Product.model = Printer.model
    WHERE Product.maker = 'B'

8)  SELECT Product.maker FROM Product
    INNER JOIN PC ON Product.model = PC.model
    EXCEPT
    SELECT Product.maker FROM Product
    INNER JOIN Laptop ON Product.model = Laptop.model
    
9)  SELECT DISTINCT(Product.maker) FROM Product
    INNER JOIN PC ON Product.model = PC.model
    WHERE speed >= 450

10) SELECT DISTINCT model, price
    FROM Printer
    WHERE price = (SELECT MAX(price)
    FROM Printer)
    
11) SELECT AVG(speed) FROM PC

12) SELECT AVG(speed) FROM Laptop WHERE price > 1000

13) SELECT AVG(PC.speed) FROM Product
    INNER JOIN PC ON Product.model = PC.model
    WHERE Product.maker = 'A'

14) SELECT Classes.class, Ships.name, Classes.country FROM Ships
    INNER JOIN Classes ON Ships.class = Classes.class
    WHERE numGuns >= 10
    
15) SELECT hd FROM PC
    GROUP BY hd 
    HAVING COUNT(model)>=2
    
16) SELECT A.model, B.model, A.speed, A.ram 
    FROM PC as A
    CROSS JOIN PC as B
    WHERE A.speed = B.speed AND A.ram = B.ram AND A.model > B.model

17) SELECT product.type, laptop.model, laptop.speed FROM laptop
    INNER JOIN product on laptop.model = product.model
    WHERE laptop.speed < (SELECT MIN(speed) FROM PC)
    
18) SELECT Product.maker, min(printer.price) FROM Product
INNER JOIN Printer ON Product.model = Printer.model
WHERE printer.color = 'y' 
GROUP BY product.maker

19) SELECT product.maker, avg(screen) FROM product
    INNER JOIN laptop ON product.model = laptop.model
    GROUP BY product.maker

20) SELECT product.maker, count(PC.model) as Count_Model
    FROM product 
    INNER JOIN PC ON product.model = PC.model
    GROUP BY product.maker HAVING COUNT(PC.model) >= 3

21) SELECT product.maker, max(pc.price) as max_price FROM product
    INNER JOIN PC ON product.model = PC.model
    GROUP BY product.maker

22) SELECT speed, avg(price) FROM PC
    WHERE speed>600
    GROUP BY speed

23) SELECT product.maker FROM product
    INNER JOIN PC On product.model = pc.model
    WHERE pc.speed >=750
    INTERSECT
    SELECT product.maker FROM product
    INNER JOIN laptop On product.model = laptop.model
    WHERE  laptop.speed >=750
    
24) SELECT t.model FROM (SELECT DISTINCT model, price FROM PC
    WHERE price = (SELECT MAX(price) FROM PC)
    UNION ALL
    SELECT DISTINCT model, price FROM laptop
    WHERE price = (SELECT MAX(price) FROM Laptop)
    UNION ALL
    SELECT DISTINCT model, price FROM printer
    WHERE price = (SELECT MAX(price) FROM printer)) as t

    WHERE t.price = (SELECT max(t.price) FROM (SELECT DISTINCT model, price FROM PC
    WHERE price = (SELECT MAX(price) FROM PC)
    UNION ALL
    SELECT DISTINCT model, price FROM laptop
    WHERE price = (SELECT MAX(price) FROM Laptop)
    UNION ALL
    SELECT DISTINCT model, price FROM printer
    WHERE price = (SELECT MAX(price) FROM printer)) as t)
    
25) SELECT t.maker FROM (SELECT product.maker, PC.ram, PC.speed FROM product
    INNER JOIN PC on product.model = PC.model
    WHERE PC.ram = (SELECT min(ram) FROM PC)) as t
    WHERE t.speed = (SELECT max(speed) FROM (SELECT product.maker, PC.ram, PC.speed FROM product
    INNER JOIN PC on product.model = PC.model
    WHERE PC.ram = (SELECT min(ram) FROM PC)) as t)
    INTERSECT
    SELECT product.maker FROM product
    INNER JOIN printer ON product.model = printer.model

26) SELECT AVG(t.price) FROM (
    SELECT price FROM product
    INNER JOIN PC on product.model = PC.model
    WHERE product.maker = 'A'
    UNION ALL
    SELECT price FROM product
    INNER JOIN laptop on product.model = laptop.model
    WHERE product.maker = 'A') as t

27) SELECT product.maker, AVG(PC.hd) as Avg_hd FROM product
    INNER JOIN PC ON product.model = PC.model
    WHERE product.maker IN (
    SELECT DISTINCT t1.maker 
    FROM (SELECT product.maker FROM product
    INNER JOIN PC ON product.model = PC.model) as t1
    INTERSECT
    SELECT DISTINCT t2.maker 
    FROM (SELECT product.maker FROM product
    INNER JOIN printer ON product.model = printer.model) as t2
    )
    GROUP BY product.maker

28) SELECT COUNT(product.maker) as qty FROM Product
    Group by product.maker
    HAVING count(model) = 1

29) 